### Installations of requirements

Note: Please restart notebook if you are running this cell. You do not have to rerun this cell everytime, it needs to be only when you need to use a different version of IDR-py library.

In [ ]:
!pip2 install --upgrade 'git+https://github.com/bramalingam/idr-py@cleanup_library'

### Import libraries and global variables

In [ ]:
import pandas

from ipywidgets import widgets, interact, fixed
from functools import wraps
from IPython.display import display, HTML

from idr.connections import connection, create_http_session
from idr.externalDBs import genes_of_interest_go
from idr.widgets import textbox_widget
from idr.widgets import dropdown_widget
from idr.attributes import get_phenotypes_for_genelist, get_similar_genes

from idr.visualizations import plot_idr_attributes, plot_string_interactions
from idr.externalDBs import genes_of_interest_from_string

import seaborn as sns
import matplotlib.pyplot as plt

## Querying

### Variables:

In [ ]:
idr_url = textbox_widget('http://idr.openmicroscopy.org', 'Enter IDR URL', 'IDR url:', True)
idr_username = textbox_widget('', 'Enter Username', 'IDR username:', True)
idr_password = textbox_widget('', 'Enter Password', 'IDR password:', True)
organisms_list = ['Homo sapiens', 'Saccharomyces cerevisiae']
org_sel = dropdown_widget(organisms_list, 'Select Organism:', True)
tax_id = textbox_widget('9606', 'Enter Taxonomy Id for Organism', "Taxonomy Id:", True)
go_term = textbox_widget('', 'Enter GO Id', 'Gene Ontology Id:', True)
manual_gene_list = textbox_widget('','Comma seperated gene symbols', 'Manual Gene List:', True)

### Import query list

In [ ]:
go_gene_list = []
if go_term.value.split(",") != ['']:
    go_gene_list = genes_of_interest_go(go_term.value, tax_id.value)
else:
    print 'Please enter a valid Gene Ontology Id'
manual_list = manual_gene_list.value.split(",")
if manual_list != ['']:
    go_gene_list = list(set(go_gene_list + manual_list))
print "Query list of genes:",go_gene_list

### Query IDR for Phenotypes

In [ ]:
session = create_http_session(idr_url.value)
organism = org_sel.value
idr_base_url = idr_url.value

[query_genes_dataframe, screen_to_phenotype_dictionary] = get_phenotypes_for_genelist(idr_base_url, session, go_gene_list, organism)
display(HTML(query_genes_dataframe.to_html( escape=False)))

### Get Other Genes from the phenotypes

In [ ]:
hostname = str(idr_url.value[idr_url.value.index('//')+2:])
username = str(idr_username.value)
password = str(idr_password.value)

conn = connection(hostname, username, password, 4064)
try:
    query_genes_list = list(query_genes_dataframe['Value'])
    [similar_genes, overlap_genes] = get_similar_genes(conn, query_genes_list, screen_to_phenotype_dictionary)
    overlap_genes_dataframe = pandas.DataFrame.from_dict(overlap_genes, orient='index')
    display(HTML("<strong>Query Genes:</strong>"))
    display(HTML(overlap_genes_dataframe.to_html( escape=False)))
    
    similar_genes_dataframe = pandas.DataFrame.from_dict(similar_genes, orient='index')
    display(HTML("<strong>Similar Genes:</strong>"))
    display(HTML(similar_genes_dataframe.to_html( escape=False)))
finally:
    conn.close()    

## Visualisation

### Plot Query Genes

In [ ]:
%matplotlib inline
filter_by_category=widgets.Dropdown(description='Filter_by_category', options=['Phenotypes','Screens'])
threshold_for_category = widgets.IntSlider(description='Threshold_for_category', min=1,max=10,step=1,value=1)
threshold_for_plot = widgets.IntSlider(description='Threshold_for_plot', min=1,max=10,step=1,value=1)
@interact(primary_dictionary=fixed(overlap_genes), secondary_dictionary=fixed(overlap_genes), plot_title=fixed('Query Genes'), filter_by_category = filter_by_category, threshold_for_category = threshold_for_category, threshold_for_plot = threshold_for_plot)
@wraps(plot_idr_attributes)
def myfun(**kwargs):
    global screenids_removed, phenotypes_removed, genes_of_interest
    [screenids_removed, phenotypes_removed, genes_of_interest] = plot_idr_attributes(**kwargs)

### Plot Similar Genes

In [ ]:
for ids in screenids_removed:
    if ids in similar_genes:
        del similar_genes[ids]

@interact(primary_dictionary=fixed(similar_genes), secondary_dictionary=fixed(overlap_genes), plot_title=fixed('Similar genes'), filter_by_category = fixed(filter_by_category.value), threshold_for_category = fixed(threshold_for_category.value),threshold_for_plot = widgets.IntSlider(min=1,max=10,step=1,value=5))
@wraps(plot_idr_attributes)
def myfun(**kwargs):
    global screenids_removed, phenotypes_removed, genes_of_interest
    [screenids_removed, phenotypes_removed, genes_of_interest] = plot_idr_attributes(**kwargs)
    
similar_genes_list = genes_of_interest

### Get String Interactions

### Plot string interactions between similar genes and query genes/similar genes

In [ ]:
similar_genes_list = list(set(genes_of_interest) - set(go_gene_list))
genes_of_interest1 = list(set(go_gene_list + similar_genes_list))
interactions_dataframe = genes_of_interest_from_string(genes_of_interest1, 1, tax_id.value)

print 'Primary Interactors:'
df = plot_string_interactions(go_gene_list, similar_genes_list, interactions_dataframe)

primary_genes = list(df.columns.values)
secondary_genes = set(similar_genes_list) - set(primary_genes)
print 'Secondary Interactors:'
df = plot_string_interactions(secondary_genes, primary_genes, interactions_dataframe)